In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from pyinverse.grid import FFTRegular, RFFTRegular, RegularGrid
from pyinverse.ellipse import Ellipse, raster_ellipse

In [3]:
e1 = Ellipse(1, 0.69, 0.92, 0, 0, 0)
e2 = Ellipse(-0.8, 0.6624, 0.8740, 0, -0.0184, 0)
e10 = Ellipse(0.1, 0.0230, 0.0460, 0.06, -0.605, 0)

In [4]:
e1.bounds

(-0.69, -0.92, 0.69, 0.92)

In [5]:
N1 = N2 = 32
grid = RegularGrid(FFTRegular(N1, d=1/2), FFTRegular(N2, d=1/2))

In [6]:
fig, ax = plt.subplots()
im = ax.imshow(raster_ellipse(e1, grid, oversample=1, doall=True), 
               extent=grid.extent)
fig.colorbar(im);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
fig, ax = plt.subplots()
im = ax.imshow(raster_ellipse(e1, grid, oversample=1, doall=False), 
               extent=grid.extent)
fig.colorbar(im);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fig, ax = plt.subplots()
im = ax.imshow(raster_ellipse(e1, grid, oversample=1, doall=True) - raster_ellipse(e1, grid, oversample=1, doall=False), 
               extent=grid.extent)
fig.colorbar(im);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig, ax = plt.subplots()
im = ax.imshow(raster_ellipse(e1, grid, oversample=4, doall=True), 
               extent=grid.extent)
fig.colorbar(im);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig, ax = plt.subplots()
im = ax.imshow(raster_ellipse(e1, grid, oversample=4, doall=False), 
               extent=grid.extent)
fig.colorbar(im);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
fig, ax = plt.subplots()
im = ax.imshow(raster_ellipse(e1, grid, oversample=4, doall=True) - raster_ellipse(e1, grid, oversample=4, doall=False), 
               extent=grid.extent)
fig.colorbar(im);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …